In [2]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
def get_data_carlsberg(url):
    
    
    """
    This function will collect the grant information provided from the Carlsberg foundation grant database with the url: "https://www.carlsbergfondet.dk/det-har-vi-stoettet/"
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    number_l = []
    year_l = []
    rec_l = []
    amount_l = []
    desc = []
    institution_l = []
    project_category_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url ("https://www.carlsbergfondet.dk/det-har-vi-stoettet/")
    return: Dataframe with text extracted
    
    """
    
    number_l = []
    year_l = []
    rec_l = []
    amount_l = []
    desc = []
    institution_l = []
    project_category_l = []
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    

    
    
    for i in range(1,2350):

        try:
            if i % 16 == 0:
                ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/main/section[2]/div/div/div[8]/div/div/button")).perform()  #move to first line for click
                element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"/html/body/main/section[2]/div/div/div[8]/div/div/button/span")))
                element.click()
                time.sleep(5)
            
            ActionChains(driver).move_to_element(driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]")).perform()  #move to extract
            time.sleep(6)
            
            number = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/p[1]").text
            project_category = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/p[2]").text
            description = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/h2").text
            receiver = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/div[1]/p[2]").text
            institution = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/div[2]").text
            amount = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/div[3]").text
            year = driver.find_element(By.XPATH, f"/html/body/main/section[2]/div/div/div[7]/div/div[2]/a[{i}]/div[4]").text
            
            number_l.append(number)
            project_category_l.append(project_category)
            desc.append(description)
            rec_l.append(receiver)
            institution_l.append(institution)
            amount_l.append(amount)
            year_l.append(year)

            
            
        except NoSuchElementException: #avoid programme crashing
            print("ended loop")

 

    df_carls = pd.DataFrame(list(zip(year_l, rec_l, amount_l, desc, institution_l, number_l, project_category_l)), columns=["Year",'Reciever','Grant size (DKK)', 'Description', "Institution", "Grant Number", "Project Category"])

    return df_carls

biglist = get_data_carlsberg("https://www.carlsbergfondet.dk/det-har-vi-stoettet/")
pd.set_option('display.max_columns', None)

biglist.to_csv('carlsberg_data.csv', index=True)